To Do :
- Add in live data example 
- Plot prediction graph 
- Plot loss and validation curve

Bug to slove :
- Error in inverse scale transform the predict output 

# Seq2Seq Multivariate Input

- **Encoder** is use to accept the multivariate input from the time series 
- **Decoder** will receive the hidden weight from the encoder state and the other time series to make a prediction 
- For Example : Given that 3 time series (3 column x-feature) , the encoder will receive all the 3 time series input ,where the decoder will receive the hidden weight from the encoder and the other 2 time series data(n-features) to make 1 time series prediction.The next LSTM unit encoder will take the output from previous unit to feed in as input.

<div>
<img src="../seq2seq.jpg"/>
</div>

In [1]:
#  import libary
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import random
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [18]:
# Hyperparameter
test_data_size =40
seq_length = 2
labels_length = 3
learning_rate = 1e-2
weight_decay = 1e-5

## Synthetic data

In [3]:
in_seq1 = np.array([x for x in range(0, 600, 10)])
in_seq2 = np.array([x for x in range(5, 605, 10)])
out_seq = np.array([in_seq1[i] + in_seq2[i] for i in range(len(in_seq1))])
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(in_seq2), 1))

In [4]:
dataset = np.hstack((in_seq1, in_seq2,out_seq))
dataset= pd.DataFrame(dataset)
dataset.columns =["Time Series 1" ,"Time Series 2","Time Series 3"]


In [5]:
train_data = dataset.iloc[1:test_data_size]
test_data = dataset.iloc[test_data_size::]
print("train_data.shape"+str(train_data.shape))
print("test_data.shape"+str(test_data.shape))

train_data.shape(39, 3)
test_data.shape(20, 3)


## Data Normalization 

In [6]:
scaler = MinMaxScaler(feature_range=(-1, 1))
train_data_normalized = scaler.fit_transform(train_data)
test_data_normalized = scaler.fit_transform(test_data)
train_data_normalized.shape

(39, 3)

In [7]:
test_data_normalized.shape

(20, 3)

## Data Sequencing 

In [19]:
def sliding_windows(data, seq_length,labels_length):
    x = []
    y = []
    z = []

    for i in range(len(data)-(seq_length+labels_length)):
        _x = data[i:(i+seq_length),:]
        _y = data[(i+seq_length):(i+seq_length+labels_length),0:1]
        _z  = data[(i+seq_length):(i+seq_length+labels_length),1:]
        x.append(np.array(_x))
        y.append(np.array(_y))
        z.append(np.array(_z))

    return x,y,z

In [20]:
train_X, train_y,train_features = sliding_windows(train_data_normalized, seq_length,labels_length)
valid_X, valid_y,valid_features = sliding_windows(test_data_normalized,seq_length,labels_length)
print("train X  has:", len(train_X) , "data")
print("train labels  has:", len(train_y) , "data")
print("validiation  X  has:", len(valid_X) , "data")
print("Validiation  labels  has:" ,len(valid_y) , "data")

train X  has: 34 data
train labels  has: 34 data
validiation  X  has: 15 data
Validiation  labels  has: 15 data


## Data Transformation

In [21]:
trainX =torch.Tensor(train_X)
trainy = torch.Tensor(train_y)
train_features = torch.Tensor(train_features)
validX = torch.Tensor(valid_X)
validy= torch.Tensor(valid_y)
valid_features = torch.Tensor(valid_features)


print ("trainX shape is:",trainX.size())
print ("trainy shape is:",trainy.size())
print ("train features  shape is:",train_features.size())
print ("validX shape is:",validX.size())
print ("validy shape is:",validy.size())
print ("valid features  shape is:",valid_features.size())

trainX shape is: torch.Size([34, 2, 3])
trainy shape is: torch.Size([34, 3, 1])
train features  shape is: torch.Size([34, 3, 2])
validX shape is: torch.Size([15, 2, 3])
validy shape is: torch.Size([15, 3, 1])
valid features  shape is: torch.Size([15, 3, 2])


## Seq2Seq Model Configuration

In [22]:
import torch
import torch.nn as nn


class Encoder(nn.Module):
#     Configure the encoder
    def __init__(self, seq_len, n_features, hidden_dim=64):
        super(Encoder, self).__init__()

        self.seq_len, self.n_features = seq_len, n_features
        self.hidden_dim = hidden_dim
        self.num_layers = 3
        self.rnn1 = nn.LSTM(
            input_size=n_features,
            hidden_size=self.hidden_dim,
            num_layers=3,
            batch_first=True,

        )
# Receive trainX input and return hidden & cell information 
    def forward(self, x):
        x = x.reshape((1, self.seq_len, self.n_features))

        h_1 = torch.zeros(
            self.num_layers, x.size(0), self.hidden_dim)

        c_1 = torch.zeros(
            self.num_layers, x.size(0), self.hidden_dim)

        x, (hidden, cell) = self.rnn1(x, (h_1, c_1))

        # return hidden_n.reshape((self.n_features, self.hidden_dim))
        return hidden, cell


class Decoder(nn.Module):
#     Configure the decoder
    def __init__(self, seq_len, input_dim=64, n_features=1):
        super(Decoder, self).__init__()

        self.seq_len, self.input_dim = seq_len, input_dim
        self.hidden_dim, self.n_features = input_dim, n_features

        self.rnn1 = nn.LSTM(
            input_size=n_features,
            hidden_size=input_dim,
            num_layers=3,
            batch_first=True,

        )

        self.output_layer = nn.Linear(self.hidden_dim, n_features)
#  Receive Encoder Hidden and Cell 
    def forward(self, x, input_hidden, input_cell):
#         Reshape to ensure output 1 time series
        x = x.reshape((1, 1, self.n_features))
        # print("decode input",x.size())

        x, (hidden_n, cell_n) = self.rnn1(x, (input_hidden, input_cell))

        x = self.output_layer(x)
        return x, hidden_n, cell_n


class Seq2Seq(nn.Module):

    def __init__(self, seq_len, n_features,output_length,hidden_dim):
        super(Seq2Seq, self).__init__()

        self.encoder = Encoder(seq_len, n_features, hidden_dim)
        self.n_features = n_features
        self.output_length = output_length
        self.decoder = Decoder(seq_len, hidden_dim, n_features)

    def forward(self, x, prev_y, features):

        hidden, cell = self.encoder(x)

        # Prepare place holder for decoder output
        targets_ta = []
        # prev_output become the next input to the LSTM cell
        dec_input = prev_y

        # dec_input = torch.cat([prev_output, curr_features], dim=1)

        # itearate over LSTM - according to the required output days
        for out_days in range(self.output_length):

            prev_x, prev_hidden, prev_cell = self.decoder(dec_input, hidden, cell)
            hidden, cell = prev_hidden, prev_cell

            prev_x = prev_x[:, :, 0:1]
#             print("preve x shape is:",prev_x.size())
            # print("features shape is:",features[out_days+1].size())

            if out_days + 1 < self.output_length:
                dec_input = torch.cat([prev_x, features[out_days + 1].reshape(1, 1, 2)], dim=2)

            targets_ta.append(prev_x.reshape(1))

        targets = torch.stack(targets_ta)

        return targets

## Deploy Model

In [23]:
n_features = trainX.shape[2]
model = Seq2Seq(seq_length, n_features, output_length = labels_length, hidden_dim=32)
print(model)

Seq2Seq(
  (encoder): Encoder(
    (rnn1): LSTM(3, 32, num_layers=3, batch_first=True)
  )
  (decoder): Decoder(
    (rnn1): LSTM(3, 32, num_layers=3, batch_first=True)
    (output_layer): Linear(in_features=32, out_features=3, bias=True)
  )
)


## Weight Initialization

In [24]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)


model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (rnn1): LSTM(3, 32, num_layers=3, batch_first=True)
  )
  (decoder): Decoder(
    (rnn1): LSTM(3, 32, num_layers=3, batch_first=True)
    (output_layer): Linear(in_features=32, out_features=3, bias=True)
  )
)

In [25]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=weight_decay)
criterion = torch.nn.MSELoss()

## Train Model

In [26]:
def train_model(model, TrainX, Trainy, ValidX, Validy, seq_length, n_epochs):
    history = dict(train=[], val=[])
    
    for epoch in range(n_epochs):
    
        train_losses = []
        validate_value= list()
        for i in range(len(TrainX)):
            seq_inp = TrainX[i, :, :]
            seq_true = Trainy[i, :, :]
            features = train_features[i, :, :]

            optimizer.zero_grad()

            seq_pred_train = model(seq_inp, seq_inp[seq_length - 1:seq_length, :], features)

            loss = criterion(seq_pred_train, seq_true)

            loss.backward()

            optimizer.step()

            train_losses.append(loss.item())

        val_losses = []
        model = model.eval()
        with torch.no_grad():
            for i in range(len(validX)):
                seq_inp = ValidX[i, :, :]
                seq_true = Validy[i, :, :]
                features = valid_features[i, :, :]

                seq_pred_validate = model(seq_inp, seq_inp[seq_length - 1:seq_length, :], features)

                loss = criterion(seq_pred_validate, seq_true)
                val_losses.append(loss.item())
                validate_value.append(seq_pred_validate)

        train_loss = np.mean(train_losses)
        val_loss = np.mean(val_losses)

        history['train'].append(train_loss)
        history['val'].append(val_loss)
        
    
    return history,validate_value
    

In [27]:
history,y_test_pred = train_model(
    model,
    trainX, trainy,
    validX, validy,
    seq_length,
    n_epochs=1,  # Train for few epochs as illustration,

)

In [28]:
history["train"]

[0.07222532447571318]

In [29]:
history["val"]

[0.8053216607309878]

Source : https://www.kaggle.com/omershect/learning-pytorch-seq2seq-with-m5-data-set/comments